In [25]:
import os                                             # Operational System para manipulação de arquivos.
import cv2                                            # OpenCV para manipulação de imagens.
import random
import numpy as np                                    # Numpy para manipulação de matrizes e arrays.
import matplotlib.pyplot as plt                       # Pyplot para plotagem de gráficos e imagens.
from sklearn.metrics import confusion_matrix          # Scikit-Learn para plotar a matriz de confusão

from tensorflow.keras import layers                   # Módulo de camadas do Keras
from tensorflow.keras import callbacks                # Módulo de callbacks do Keras
from tensorflow.keras import optimizers               # Módulo de otimizadores do Keras
from tensorflow.keras.models import load_model        # Função para carregar um modelo salvo
from tensorflow.keras.models import Sequential        # Classe de modelos sequenciais para construir as redes neurais.
from tensorflow.keras.applications import VGG16       # Classe de modelos sequenciais para construir as redes neurais.

from google_drive_downloader import GoogleDriveDownloader as gdd

# ImageDataGenerator, utilizado para carregar imagens em tempo de execução
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!ls


drive  model.h5  sample_data


In [28]:
val_datagen   = ImageDataGenerator( rescale = 1./255 )
val_generator = val_datagen.flow_from_directory("/content/drive/MyDrive/Chest Cancer Detection Resized/Data/validation resized", target_size = (250, 250),
                                                 batch_size  = 10)

Found 72 images belonging to 4 classes.


In [29]:
# Atributo do generator que fornece o número de amostras detectadas
val_samples = val_generator.samples 
print(val_samples, "amostras detectadas")

# Atributo do generator que fornece o mapeamento de classe para índice 
# Repare que os índices são definidos pelo generator com base nos diretórios de arquivos em ordem alfabética
class_to_idx_dict = val_generator.class_indices
print( "Mapeamento Classes -> Índices:", class_to_idx_dict )

# Construção de um novo dicionário que inverte o mapeamento
idx_to_class_dict = { v: k for k, v in class_to_idx_dict.items() }
print( "Mapeamento Índices -> Classes:", idx_to_class_dict )

72 amostras detectadas
Mapeamento Classes -> Índices: {'Resized adenocarcinoma': 0, 'Resized large.cell.carcinoma': 1, 'Resized normal': 2, 'Resized squamous.cell.carcinoma': 3}
Mapeamento Índices -> Classes: {0: 'Resized adenocarcinoma', 1: 'Resized large.cell.carcinoma', 2: 'Resized normal', 3: 'Resized squamous.cell.carcinoma'}


In [56]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,        # normalizando as imagens
    rotation_range = 20,       # Rotação aleatória de até 20°
    width_shift_range = 0.2,   # Translação horizontal de até 20% da largura
    height_shift_range = 0.2,  # Translação vertical de até 20% da altura
    zoom_range = 0.2,          # Zoom aleatório de até 20%
    shear_range = 0.1,         # Deformação de 10%
    horizontal_flip = True,    # Espelhamento horizontal aleatório
    vertical_flip = False,     # Espelhamento vertical aleatório
    fill_mode = "nearest")     # Preenchimentod e buracos pelo pixel mais próximo

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Chest Cancer Detection Resized/Data/train resized", target_size = (250, 250),
    batch_size  = 10)

# Atributo do generator que fornece o número de amostras detectadas
train_samples = train_generator.samples 
print(train_samples)

Found 601 images belonging to 4 classes.
601


In [57]:
def print_vgg16_summary( shape ):
    conv_base = VGG16(include_top = False, weights = "imagenet", input_shape = shape)
    print( conv_base.summary() )
    return

print_vgg16_summary( shape = (250, 250, 3) )

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 250, 250, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 250, 250, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 125, 125, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 125, 125, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 125, 125, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 62, 62, 128)       0     

In [58]:
def build_model( input_shape, n_outputs ):
    # Base convolucional
    conv_base = VGG16(include_top = False, weights = "imagenet", input_shape = input_shape)
    conv_base.trainable = False
    
    rede = Sequential()
    rede.add( conv_base )
    rede.add( layers.Flatten() )
    rede.add( layers.Dense( 256, activation = "relu" ) )
    rede.add( layers.Dense(   4, activation = "softmax" ) )
    
    return rede

model = build_model( (250, 250, 3), 1 )
model.compile(optimizer=optimizers.Adam(lr=1e-4), 
              loss="categorical_crossentropy", 
              metrics=["acc"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 256)               6422784   
                                                                 
 dense_7 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 6,423,812
Non-trainable params: 14,714,688
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [59]:
# definindo todas as camadas do modelo como treináveis 
model.trainable = True
# obtendo as camadas convolucionais da arquitetura VGG16
conv_base = model.get_layer("vgg16")

# criando uma flag de estado de treinamento e inicializando-a como falsa (estado não treinável) 
set_trainable = False
# para cada uma das camadas convolucionais da VGG16...
for layer in conv_base.layers:
    # se a camada for igual a 'block5_conv2'...
    if layer.name == "block3_conv3":
        # a flag de estado de treinamento muda de estado para estado treinável
        set_trainable = True
    # todas as camadas posteriores a 'block5_conv2' são configuradas como treinável 
    # e as camadas anteriores como não treináveis
    layer.trainable = set_trainable

In [60]:
conv_base = model.get_layer("vgg16")
for layer in conv_base.layers:
    status = "Treinável" if layer.trainable else "Congelada"
    print("Camada '{}' - Status: {} - Entrada: {} - Saída: {}".format(layer.name,
                                                                      status, 
                                                                      layer.input_shape, 
                                                                      layer.output_shape))

Camada 'input_8' - Status: Congelada - Entrada: [(None, 250, 250, 3)] - Saída: [(None, 250, 250, 3)]
Camada 'block1_conv1' - Status: Congelada - Entrada: (None, 250, 250, 3) - Saída: (None, 250, 250, 64)
Camada 'block1_conv2' - Status: Congelada - Entrada: (None, 250, 250, 64) - Saída: (None, 250, 250, 64)
Camada 'block1_pool' - Status: Congelada - Entrada: (None, 250, 250, 64) - Saída: (None, 125, 125, 64)
Camada 'block2_conv1' - Status: Congelada - Entrada: (None, 125, 125, 64) - Saída: (None, 125, 125, 128)
Camada 'block2_conv2' - Status: Congelada - Entrada: (None, 125, 125, 128) - Saída: (None, 125, 125, 128)
Camada 'block2_pool' - Status: Congelada - Entrada: (None, 125, 125, 128) - Saída: (None, 62, 62, 128)
Camada 'block3_conv1' - Status: Congelada - Entrada: (None, 62, 62, 128) - Saída: (None, 62, 62, 256)
Camada 'block3_conv2' - Status: Congelada - Entrada: (None, 62, 62, 256) - Saída: (None, 62, 62, 256)
Camada 'block3_conv3' - Status: Treinável - Entrada: (None, 62, 62, 256

In [61]:
model.compile(optimizer=optimizers.Adam(learning_rate = 1e-5), 
              loss = "categorical_crossentropy", 
              metrics = ["acc"])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 256)               6422784   
                                                                 
 dense_7 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 19,993,092
Non-trainable params: 1,145,408
_________________________________________________________________


In [62]:
model_checkpoint = callbacks.ModelCheckpoint("model.h5", monitor = "acc", save_best_only = True, verbose = 1)
reduce_lr_on_plateau = callbacks.ReduceLROnPlateau(monitor = "acc", factor = 0.75, patience = 3, verbose = 1)

# Repare que ao utilizar mais de 1 callback elas devem ser organizadas em uma lista
callback_list = [model_checkpoint, reduce_lr_on_plateau]

In [63]:
history = model.fit( train_generator, steps_per_epoch = 20,
                     epochs = 100, callbacks = callback_list, 
                     validation_data = val_generator, validation_steps = 20 )

model.load_weights("model.h5")

history_dict = history.history

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 1.3668 - acc: 0.3950WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 20 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: acc improved from -inf to 0.39500, saving model to model.h5
20/20 [==============================] - 6s 279ms/step - loss: 1.3668 - acc: 0.3950 - val_loss: 1.2290 - val_acc: 0.3056 - lr: 1.0000e-05
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 1.1734 - acc: 0.4950
Epoch 2: acc improved from 0.39500 to 0.49500, saving model to model.h5
20/20 [==============================] - 5s 228ms/step - loss: 1.1734 - acc: 0.4950 - lr: 1.0000e-05
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 1.0495 - acc: 0.5800
Epoch 3: acc improved from 0.49500 to 0.58000, saving model to model.h5
20/20 [=====

In [64]:
test_datagen   = ImageDataGenerator( rescale = 1./255 )
test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/Chest Cancer Detection Resized/Data/test resized", target_size = (250, 250),
                                                   batch_size  = 1, shuffle = False)

# Atributo do generator que fornece o número de amostras detectadas
test_samples = test_generator.samples

test_loss, test_acc = model.evaluate( test_generator )

print("Test Accuracy:", 100*test_acc, "%")
print("Acertos: {} - Erros: {}".format(round(test_samples * test_acc), 
                                       round(test_samples * (1-test_acc) )))

Found 315 images belonging to 4 classes.
315/315 [==============================] - 4s 13ms/step - loss: 0.3227 - acc: 0.8952
Test Accuracy: 89.52381014823914 %
Acertos: 282 - Erros: 33
